In [1]:
import matplotlib.pyplot as plt
import numpy as np
from gym.wrappers import (
    FilterObservation,
    FlattenObservation,
    RecordVideo,
    RescaleAction,
    TimeLimit,
)
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import unwrap_wrapper

from ea_train import ARESEACheetah


initializing ocelot...


In [2]:
env = ARESEACheetah(
    incoming_mode="random",
    incoming_values=None,
    misalignment_mode="random",
    misalignment_values=None,
    action_mode="direct",
    magnet_init_mode="random",
    magnet_init_values=None,
    reward_mode="differential",
    target_beam_mode="random",
    target_beam_values=None,
    w_mu_x=1.0,
    w_mu_y=1.0,
    w_on_screen=1.0,
    w_sigma_x=1.0,
    w_sigma_y=1.0,
    w_time=1.0,
)
env = RescaleAction(env, -1, 1)

check_env(env)


In [3]:
env.step(np.zeros(5))

({'beam': array([0.00103681, 0.00044352, 0.0010459 , 0.00047386], dtype=float32),
  'magnets': array([0., 0., 0., 0., 0.], dtype=float32),
  'target': array([-0.00073662,  0.00191974,  0.00131838,  0.00049668], dtype=float32),
  'incoming': array([ 1.0362302e+08,  9.7383739e-04,  5.0255920e-07, -8.2123058e-04,
         -1.8168690e-05,  4.3160302e-04,  4.9915037e-05,  4.6376302e-04,
          4.7561254e-05,  7.5433154e-06,  6.7512371e-04], dtype=float32),
  'misalignments': array([ 7.6631660e-04,  1.4502162e-03, -1.1432454e-03,  1.8725955e-03,
         -9.3780149e-04, -1.4829753e-03, -6.1949118e-05, -1.9043040e-03],
        dtype=float32)},
 1.0426414253883323,
 False,
 {'binning': array(4),
  'l1_distance': 0.003544951556250453,
  'on_screen_reward': 1,
  'pixel_size': array([1.32792e-05, 9.78760e-06]),
  'screen_resolution': array([612., 510.]),
  'time_reward': -1})